In [3]:
#Imports and Set up

import random
import pandas as pd
import numpy as np
import json 
import sys
import spotipy
import spotipy.util as util
from bokeh.plotting import figure, output_file, show, output_notebook,ColumnDataSource
from bokeh.models import HoverTool
from bokeh.colors import RGB
#Setting Up your Credientials for Spotify
username ='aidand1999'
client_id='f4af21b115c44ae0b9b2d14f50de3511'
client_secret='6d8734442894429188e844b9362b45e0'
redirect_uri='http://localhost:8080'

In [4]:
#Reading my User Data
f = open('StreamingHistory0.json')
History0 = json.load(f)
f.close()
f = open('StreamingHistory1.json')
History1 = json.load(f)
f.close()
History = History0 + History1

In [5]:
class Artist:
  def __init__(self, name, plays):
    self.name = name
    self.plays = plays

In [6]:
class Genre:
  def __init__(self, name, plays):
    self.name = name
    self.plays = plays

In [7]:
class Song:
  def __init__(self, name, plays):
    self.name = name
    self.plays = plays

In [8]:
def Getartists(list1):
    unique_Arists = []
    for x in list1:
        if x['artistName'] not in unique_Arists:
            unique_Arists.append(x['artistName'])
    return unique_Arists 
ArtistList = Getartists(History)

Artists = []
for x in ArtistList:
    Artists.append(Artist(x,0))
    
playsperArtist = Artists;
for x in History:
    for y in playsperArtist:
        if x['artistName'] == y.name:
            y.plays = y.plays + 1

In [9]:
#for x in playsperArtist:
#    print(x.name, x.plays)

In [10]:
def GetSongs(list1):
    unique_Songs = []
    for x in list1:
        if x['trackName'] not in unique_Songs:
            unique_Songs.append(x['trackName'])
    return unique_Songs 
SongList = GetSongs(History)

Songs = []
for x in SongList:
    Songs.append(Song(x,0))
    
playsperSong = Songs;
for x in History:
    for y in playsperSong:
        if x['trackName'] == y.name:
            y.plays = y.plays + 1

In [11]:
#for x in playsperSong:
#    print(x.name,x.plays)

In [12]:
#create Np array
#finalarray = np.array([],[])
#for x in playsperSong:
#    print(x.plays,x.name)
#    temp = np.array([x.plays],[x.plays])
#    finalarray = np.append(finalarray,temp)

In [13]:
playspersong1 = []
for x in playsperSong:
    temp = [x.name,x.plays]
    playspersong1.append(temp)

df = pd.DataFrame(data=playspersong1)
df.columns = ['name','plays']
df['name'] = df['name'].astype("string")
df = df.sort_values(by=['plays'],ascending=False)

In [14]:
#Bohek Stuff
source = ColumnDataSource(df)
output_notebook()

TOOLTIPS = [
    ("Name", "@name"),
    ("Plays", "@plays"),
]

p = figure(plot_width=1000, plot_height=400,tooltips=TOOLTIPS)




# add a circle renderer with a size, color, and alpha
p.circle(x="plays",y=0,size ="plays", source = source)

# show the results
show(p)

Loading BokehJS ...

In [15]:
dftempSongs = df
dftempSongs['plays']


#for x in range(0,len(dftemp.index)-1):
#    temp = dftemp['plays'].iloc[x]
#    xvalues.append(temp)

tot = np.cumsum(dftempSongs['plays']) 
dftempSongs.insert(2, 'xvalues', tot/3, allow_duplicates = False)
    

In [16]:
#Bokeh Stuff
linesource = ColumnDataSource(dftempSongs)
circlesource = ColumnDataSource(dftempSongs.head(100))
output_notebook()
#output_file("line.html")

TOOLTIPS = [
    ("Name", "@name"),
    ("Plays", "@plays"),
]

p = figure(plot_width=1000, plot_height=1000,tooltips=TOOLTIPS,y_axis_label='Number of Plays per Song',x_axis_label='Hover for song title',title='My top played Songs on Spotify')




# add a circle renderer with a size, color, and alpha
p.circle(x="xvalues",y='plays',size =8, source = circlesource,fill_color="white")
p.line(x="xvalues",y='plays', line_width=2,source = linesource)

# show the results
show(p)

Loading BokehJS ...

In [17]:
playsperArtist1 = []
for x in playsperArtist:
    temp = [x.name,x.plays]
    playsperArtist1.append(temp)

df = pd.DataFrame(data=playsperArtist1)
df.columns = ['name','plays']
df['name'] = df['name'].astype("string")
df = df.sort_values(by=['plays'],ascending=False)


In [18]:
dftempArtist = df
dftempArtist['plays']


#for x in range(0,len(dftemp.index)-1):
#    temp = dftemp['plays'].iloc[x]
#    xvalues.append(temp)

tot = np.cumsum(dftempArtist['plays']) 
dftempArtist.insert(2, 'xvalues', tot/3, allow_duplicates = False)

In [19]:
#Bokeh Stuff
linesource = ColumnDataSource(dftempArtist)
circlesource = ColumnDataSource(dftempArtist.head(10))
output_notebook()
#output_file("line.html")

TOOLTIPS = [
    ("Name", "@name"),
    ("Plays", "@plays"),
]

p = figure(plot_width=1000, plot_height=1000,tooltips=TOOLTIPS,y_axis_label='Number of Plays per Artist',x_axis_label='Hover for Artist Name',title='My top played Artists on Spotify')




# add a circle renderer with a size, color, and alpha
p.circle(x="xvalues",y='plays',size =8, source = circlesource,fill_color="white")
p.line(x="xvalues",y='plays', line_width=2,source = linesource)

# show the results
show(p)

Loading BokehJS ...

In [20]:
#Spotify Api Needed

In [68]:
name = 'Bon Iver'
#Getting Auth Token
scope = 'user-library-read'
token = util.prompt_for_user_token(username, scope,client_id,client_secret,redirect_uri)
if token:
        sp = spotipy.Spotify(auth=token)
        
        
else:
    print("Can't get token for", username)

results = sp.search(q='artist:' + name, type='artist')
items = results['artists']['items']
current_uri = items[0]['uri']

results = sp.artist_top_tracks(current_uri)

for track in results['tracks'][:10]:
    print('track    : ' + track['name'])
    print('audio    : ' + track['preview_url'])
    print('cover art: ' + track['album']['images'][0]['url'])
    print()

track    : Skinny Love
audio    : https://p.scdn.co/mp3-preview/9964fbfa8b9d85d3b4edcd5a47c6386f0eadee92?cid=f4af21b115c44ae0b9b2d14f50de3511
cover art: https://i.scdn.co/image/ab67616d0000b2730b1993db92eecc2dd5021293

track    : Hey, Ma
audio    : https://p.scdn.co/mp3-preview/625ef5d640863ea2570bf591bb30b482ff83b1aa?cid=f4af21b115c44ae0b9b2d14f50de3511
cover art: https://i.scdn.co/image/ab67616d0000b2735cc90d375ab1e4423615654c

track    : Holocene
audio    : https://p.scdn.co/mp3-preview/a07bbe36e9ce8fc15114becac8876d3a20517cf2?cid=f4af21b115c44ae0b9b2d14f50de3511
cover art: https://i.scdn.co/image/ab67616d0000b2736e68b2cb5583d891095d8c21

track    : Rosyln
audio    : https://p.scdn.co/mp3-preview/f06c81d71ac4cfd9a96c34c1e8190cf3334bec11?cid=f4af21b115c44ae0b9b2d14f50de3511
cover art: https://i.scdn.co/image/ab67616d0000b2738cdc3315bbb5331eddf6a21d

track    : PDLIF
audio    : https://p.scdn.co/mp3-preview/8289ad24b7fe5ae2f038592262c1f4ab30c6279f?cid=f4af21b115c44ae0b9b2d14f50de3511


In [43]:
len(dftempArtist)

1638

In [44]:
dfArtist10 = dftempArtist

In [45]:
def GetGenres (name):
    
    #Getting Auth Token
    if token:
        sp = spotipy.Spotify(auth=token)
        
        
    else:
        print("Can't get token for", username)

    results = sp.search(q='artist:' + name, type='artist')


    return results['artists']['items'][0]['genres']

In [37]:
print(GetGenres("Loyle Carner"))

['indie r&b', 'indie soul', 'london rap', 'uk alternative hip hop']


In [47]:
ArtistNames = dfArtist10.name
GenreData = []
for x in ArtistNames:
    try:
        temp = GetGenres(x)
        GenreData.append(temp)
        print(temp)
    except:
          print("It messed up")

    

['neo mellow', 'pop', 'pop rock', 'singer-songwriter']
['irish singer-songwriter', 'pop']
['folk-pop', 'neo mellow', 'pop', 'pop rock']
['bedroom pop', 'indie r&b', 'pop']
['chamber pop', 'eau claire indie', 'indie folk', 'melancholia', 'slow core']
['bow pop', 'compositional ambient', 'neo-classical']
['indie r&b', 'indie soul', 'london rap', 'uk alternative hip hop']
['irish hip hop']
['atl hip hop', 'hip hop', 'pop rap', 'rap']
['classic rock', 'folk', 'folk rock', 'melancholia', 'mellow gold', 'rock', 'roots rock']
['chamber pop', 'indie anthem-folk', 'indie folk', 'indiecoustica']
['british singer-songwriter', 'chamber pop', 'fingerstyle', 'folk-pop', 'indie folk', 'neo mellow', 'stomp and holler']
['indie soul']
['chicago rap', 'conscious hip hop', 'hip hop', 'pop rap', 'rap']
['british invasion', 'classic rock', 'merseybeat', 'psychedelic rock', 'rock']
['chicago rap', 'rap']
['celtic', 'irish folk']
['canadian contemporary r&b', 'pop']
['pop', 'uk pop']
['indie rock', 'modern r

['australian indie', 'australian pop']
['deep euro house', 'deep house', 'deep tropical house', 'minimal tech house', 'new french touch', 'tropical house']
['birmingham hip hop', 'grime', 'modern rock', 'uk garage']
['albuquerque indie', 'alternative rock', 'chamber pop', 'garage rock', 'indie folk', 'indie pop', 'indie rock', 'modern rock', 'portland indie', 'stomp and holler']
['british folk', 'celtic', 'celtic rock', 'irish folk', 'scottish folk']
['bebop', 'contemporary post-bop', 'cool jazz', 'hard bop', 'jazz', 'jazz fusion', 'jazz trumpet']
['baroque pop', 'chamber pop', 'freak folk', 'indie folk', 'singer-songwriter']
['folk-pop', 'indie pop', 'metropopolis', 'modern alternative rock', 'modern rock', 'pop rock', 'stomp and holler']
['chamber pop', 'indie anthem-folk', 'indie folk', 'new americana', 'stomp and holler', 'uk americana']
['downtempo', 'electronica', 'ninja', 'nu jazz']
['electropop', 'indie pop', 'indie poptimism', 'indietronica', 'modern rock', 'pop']
['neo mellow

['texas country']
['modern rock', 'neo mellow', 'piano rock', 'pop', 'pop rock']
['folk', 'new americana']
['english indie rock', 'modern rock', 'rock', 'warrington indie']
['indie pop', 'indietronica', 'irish rock', 'modern alternative rock', 'modern rock', 'northern irish indie', 'rock']
['garage rock', 'modern rock', 'rock', 'sheffield indie']
['emo', 'modern rock', 'pop punk']
['acoustic pop', 'hollywood', 'lilith', 'neo mellow', 'pop', 'pop rock', 'post-teen pop']
['brighton indie', 'english indie rock', 'modern alternative rock', 'modern rock', 'rock']
['alternative rock', 'dance-punk', 'garage rock', 'indie rock', 'modern rock', 'new rave', 'rock', 'scottish rock']
['alternative rock', 'modern power pop', 'modern rock', 'permanent wave', 'pop rock', 'post-grunge', 'rock']
['chillwave', 'ninja']
['indie anthem-folk', 'uk contemporary r&b']
['indie anthem-folk', 'indie pop']
['alternative dance', 'garage rock', 'indie rock', 'indietronica', 'modern alternative rock', 'modern rock'

['french rock']
['french indie pop']
['uk pop']
['indie pop', 'modern alternative rock', 'modern rock', 'rock']
['french indie pop', 'french indietronica', 'new french touch']
['indie garage rock']
['indie anthem-folk']
['chillhop', 'lo-fi beats']
['chillhop', 'lo-fi beats']
['electropop', 'indie folk', 'indie pop', 'indie poptimism', 'indietronica', 'modern rock', 'nyc pop', 'stomp and holler', 'vapor soul']
['british singer-songwriter', 'chamber pop', 'indie anthem-folk', 'indie folk']
['lo-fi beats']
['chillhop', 'jazz boom bap', 'lo-fi beats']
['chillhop', 'lo-fi beats']
['chillhop', 'jazz boom bap', 'lo-fi beats']
['french indie pop', 'french indietronica']
['dance pop', 'eurodance', 'europop']
['chamber pop', 'la indie', 'metropopolis', 'shimmer pop', 'stomp and holler']
['broadway', 'hollywood', 'pop', 'show tunes']
['chicago rap', 'melodic rap']
['pop', 'pop rap']
['alternative dance', 'dance-punk', 'english indie rock', 'garage rock', 'indie rock', 'modern rock', 'new rave']
[

['alternative country', 'alternative rock', 'anti-folk', 'folk', 'folk punk', 'folk rock', 'permanent wave', 'roots rock', 'singer-songwriter', 'traditional folk', 'uk americana']
['french indie pop', 'french indietronica', 'new french touch']
['bossa nova', 'brazilian jazz', 'choro', 'forro', 'jazz guitar', 'mpb', 'samba', 'velha guarda', 'violao']
['chillhop', 'lo-fi beats']
['brostep', 'catstep', 'edm', 'electro house', 'gaming edm']
['afrofuturism', 'alternative r&b', 'art pop', 'escape room', 'indie soul']
['dance pop', 'hyphy', 'pop rap', 'rap', 'southern hip hop', 'trap']
['indie folk', 'indie rock', 'louisville indie', 'new americana', 'shiver pop', 'stomp and holler']
['future funk', 'hardvapour', 'vaporwave']
['jazz funk', 'smooth jazz']
['dance pop', 'dance rock', 'disco', 'europop', 'new romantic', 'new wave pop']
['australian dance', 'australian pop', 'dance pop', 'pop']
['gqom']
['alternative hip hop', 'conscious hip hop', 'east coast hip hop', 'hardcore hip hop', 'hip ho

...

[]
['chillhop', 'jazz boom bap', 'lo-fi beats']
['electronica', 'french indie pop', 'french indietronica', 'new french touch']
['modern rock', 'neo mellow', 'pop', 'pop rock']
['chamber pop', 'freak folk', 'indie rock']
['english indie rock', 'indie anthem-folk', 'indie poptimism', 'indie surf', 'modern alternative rock']
['chillhop', 'lo-fi beats']
['lo-fi beats']
['chillhop', 'jazz boom bap', 'lo-fi beats']
['british singer-songwriter', 'chamber pop', 'indie anthem-folk', 'indie folk', 'indiecoustica', 'new americana', 'stomp and holler']
['folk-pop', 'new americana']
[]
['dance pop', 'deep tropical house', 'edm', 'electro house', 'pop', 'pop edm', 'tropical house']
['celtic', 'irish country', 'irish folk']
['french indie pop']
['swedish alternative rock']
['french indie pop', 'french indietronica', 'new french touch']
['alternative rock', 'permanent wave', 'pop rock', 'rock']
['electropop', 'indie pop', 'modern rock', 'pop']
['alternative dance', 'dance-punk', 'disco house', 'electr

['canadian pop', 'indie quebecois']
['album rock', 'art rock', 'classic rock', 'dance rock', 'mellow gold', 'new wave', 'permanent wave', 'rock', 'soft rock']
[]
['acoustic pop', 'indie folk', 'indiecoustica', 'neo mellow', 'new americana', 'roots worship', 'stomp and holler']
['indie anthem-folk', 'indiecoustica']
[]
[]
['alternative r&b', 'indie r&b', 'indie soul']
[]
['bedroom pop', 'indie garage rock', 'uk contemporary r&b']
[]
['australian reggae fusion', 'reggae fusion']
['downtempo', 'livetronica', 'new tribe', 'psychedelic trance', 'psychill']
[]
['alternative r&b', 'atl hip hop', 'hip hop', 'melodic rap', 'pop', 'pop rap', 'rap', 'trap']
['nz folk']
['swedish idol pop', 'swedish pop']
['grupera', 'gruperas inmortales', 'nu-cumbia', 'regional mexican']
['indie folk', 'indie pop', 'indie rock', 'new jersey indie', 'small room']
['chill beats']
['alternative pop rock', 'pop']
['dutch pop', 'indie jazz', 'indie soul']
[]
['soft rock', 'sunshine pop']
['chicago soul', 'classic soul

['country rock', 'folk', 'folk rock', 'indie folk', 'mellow gold', 'new americana', 'roots rock', 'singer-songwriter', 'traditional folk']
[]
['american folk revival', 'appalachian folk', 'folk', 'folk rock', 'traditional blues', 'traditional folk']
['american folk revival', 'brill building pop', 'bubblegum pop', 'folk', 'folk rock', 'mellow gold', 'soft rock', 'traditional folk']
['american folk revival', 'folk', 'folk rock', 'traditional folk']
['irish singer-songwriter']
['cosmic american', 'country rock', 'folk', 'folk rock', 'mellow gold', 'psychedelic rock', 'roots rock', 'singer-songwriter', 'soft rock', 'traditional folk']
['indie game soundtrack', 'scorecore', 'soundtrack', 'video game music']
[]
['ambient folk', 'british singer-songwriter', 'chamber pop', 'indie anthem-folk', 'indie folk']
['cosmic post-rock', 'instrumental post-rock', 'post-rock']
['australian americana', 'funk']
['british post-rock', 'instrumental post-rock', 'post-rock']
['chamber pop', 'indie game soundtr

['focus']
['background music', 'focus']
['ambient', 'art pop', 'art rock', 'compositional ambient', 'drone', 'fourth world']
['soundtrack']
[]
['ambient', 'compositional ambient', 'drift', 'drone', 'nu age', 'warm drone']
[]
['compositional ambient', 'focus', 'neo-classical']
['background music', 'focus', 'neo-classical']
['background music']
['focus']
[]
['focus']
['calming instrumental']
['indiana indie', 'indie folk', 'indie garage rock', 'indie rock']
['abstract idm', 'ambient', 'compositional ambient', 'drift', 'drone', 'focus', 'neo-classical', 'warm drone']
['healing', 'meditation', 'native american', 'new age']
['background music']
['alternative rock', 'dream pop', 'melancholia', 'slow core']
['alternative rock', 'neo mellow', 'pop rock', 'post-grunge', 'rock']
['contemporary country', 'country', 'country dawn', 'country road', 'pop rock']
['alternative rock', 'mellow gold', 'neo mellow', 'pop rock', 'post-grunge', 'rock']
['celtic', 'irish country', 'irish folk']
['alternative

['canadian pop', 'dance pop', 'electropop', 'indie poptimism', 'pop', 'post-teen pop']
['modern rock', 'rock']
['candy pop', 'dance pop', 'lilith', 'neo mellow', 'pop rock', 'post-teen pop']
[]
['folk-pop', 'pop folk']
[]
['boy band', 'hollywood', 'post-teen pop']
['british indie rock', 'dance-punk', 'garage rock', 'indie rock', 'modern rock', 'new rave', 'north east england indie']
['edm', 'electropop', 'pop', 'tropical house']
['boy band', 'dance pop', 'pop', 'post-teen pop']
['boy band', 'dance pop', 'pop', 'post-teen pop', 'viral pop']
['dance pop', 'pop', 'post-teen pop', 'viral pop']
['irish folk']
['pop', 'post-teen pop']
['britpop', 'chamber pop', 'indie rock', 'modern rock', 'neo mellow', 'rock']
['dance pop', 'pop', 'post-teen pop', 'tropical house']
['broadway', 'disney', 'hollywood', 'movie tunes', 'show tunes']
['comic', 'dark cabaret', 'otacore', 'steampunk']
['dance pop', 'pop', 'pop rap', 'post-teen pop', 'urban contemporary']
[]
['antiviral pop', 'otacore']
['cartoon']

In [48]:
def Flatten(l):
    flat_list = []
    for sublist in l:
        for item in sublist:
            flat_list.append(item)
    return flat_list

In [49]:
FlattenedGenres = Flatten(GenreData)
FlattenedGenres

['neo mellow',
 'pop',
 'pop rock',
 'singer-songwriter',
 'irish singer-songwriter',
 'pop',
 'folk-pop',
 'neo mellow',
 'pop',
 'pop rock',
 'bedroom pop',
 'indie r&b',
 'pop',
 'chamber pop',
 'eau claire indie',
 'indie folk',
 'melancholia',
 'slow core',
 'bow pop',
 'compositional ambient',
 'neo-classical',
 'indie r&b',
 'indie soul',
 'london rap',
 'uk alternative hip hop',
 'irish hip hop',
 'atl hip hop',
 'hip hop',
 'pop rap',
 'rap',
 'classic rock',
 'folk',
 'folk rock',
 'melancholia',
 'mellow gold',
 'rock',
 'roots rock',
 'chamber pop',
 'indie anthem-folk',
 'indie folk',
 'indiecoustica',
 'british singer-songwriter',
 'chamber pop',
 'fingerstyle',
 'folk-pop',
 'indie folk',
 'neo mellow',
 'stomp and holler',
 'indie soul',
 'chicago rap',
 'conscious hip hop',
 'hip hop',
 'pop rap',
 'rap',
 'british invasion',
 'classic rock',
 'merseybeat',
 'psychedelic rock',
 'rock',
 'chicago rap',
 'rap',
 'celtic',
 'irish folk',
 'canadian contemporary r&b',
 'p

In [50]:
def GetUniqueGenres(list1):
    unique_Genres = []
    for x in list1:
        if x not in unique_Genres:
            unique_Genres.append(x)
    return unique_Genres 

UniqueGenres = GetUniqueGenres(FlattenedGenres)


BlankGenrePlays = []
for x in UniqueGenres:
    BlankGenrePlays.append([x,0])

playsperGenre = BlankGenrePlays;
for x in FlattenedGenres:
    for y in playsperGenre:
        if x == y[0]:
            y[1] =y[1]+1
    


In [53]:
print(playsperGenre)

[['neo mellow', 95], ['pop', 183], ['pop rock', 96], ['singer-songwriter', 41], ['irish singer-songwriter', 10], ['folk-pop', 32], ['bedroom pop', 15], ['indie r&b', 29], ['chamber pop', 144], ['eau claire indie', 6], ['indie folk', 153], ['melancholia', 18], ['slow core', 13], ['bow pop', 10], ['compositional ambient', 18], ['neo-classical', 14], ['indie soul', 41], ['london rap', 2], ['uk alternative hip hop', 15], ['irish hip hop', 6], ['atl hip hop', 16], ['hip hop', 57], ['pop rap', 66], ['rap', 69], ['classic rock', 89], ['folk', 56], ['folk rock', 76], ['mellow gold', 84], ['rock', 178], ['roots rock', 59], ['indie anthem-folk', 62], ['indiecoustica', 31], ['british singer-songwriter', 16], ['fingerstyle', 2], ['stomp and holler', 114], ['chicago rap', 12], ['conscious hip hop', 9], ['british invasion', 5], ['merseybeat', 2], ['psychedelic rock', 30], ['celtic', 28], ['irish folk', 35], ['canadian contemporary r&b', 3], ['uk pop', 18], ['indie rock', 87], ['modern rock', 137], [

In [61]:
#Bokeh for Genres
dftempGenre = pd.DataFrame(data=playsperGenre)
dftempGenre.columns = ['name','plays']
dftempGenre['name'] = dftempGenre['name'].astype("string")
dftempGenre = dftempGenre.sort_values(by=['plays'],ascending=False)


dftempGenre



tot = np.cumsum(dftempGenre['plays']) 
dftempGenre.insert(2, 'xvalues', tot/3, allow_duplicates = False)

In [67]:
#Bokeh Stuff
linesource = ColumnDataSource(dftempGenre)
circlesource = ColumnDataSource(dftempGenre.head(30))
output_notebook()
#output_file("line.html")

TOOLTIPS = [
    ("Name", "@name"),
    ("Plays", "@plays"),
]

p = figure(plot_width=1000, plot_height=1000,tooltips=TOOLTIPS,y_axis_label='Number of Plays per Genre',x_axis_label='Hover for Genre Name',title='My top played Genres on Spotify')




# add a circle renderer with a size, color, and alpha
p.circle(x="xvalues",y='plays',size =8, source = circlesource,fill_color="white")
p.line(x="xvalues",y='plays', line_width=2,source = linesource)

# show the results
show(p)

Loading BokehJS ...